# 🏋️ AI Search + Agent Service: Fitness-Fun Example 🤸

Welcome to our **AI Search + AI Agent** tutorial, where we'll:

1. **Create** an Azure AI Search index with some fitness-oriented sample data
2. **Demonstrate** how to connect that index to an Agent via the `AzureAISearchTool`
3. **Show** how to query the Agent for health and fitness info in a fun scenario (with disclaimers!)

## 🏥 Health & Fitness Disclaimer
> **This notebook is for general demonstration and entertainment purposes, NOT a substitute for professional medical advice.**
> Always seek the advice of certified health professionals.

## Prerequisites
1. A **Microsoft Azure** subscription.
2. An **Azure AI Search** resource (formerly "Cognitive Search"), with admin API key or role-based access.
3. **Python 3.8+**, plus these libraries:
   ```bash
   pip install azure-search-documents==11.4.0
   pip install azure-ai-projects azure-identity
   pip install opentelemetry-sdk azure-core-tracing-opentelemetry
   ```
4. Environment variables or a way to store your secrets:
   - `SEARCH_ENDPOINT` - The endpoint URL of your Azure AI Search service
   - `SEARCH_API_KEY` - The admin key for your Azure AI Search service
   - `PROJECT_CONNECTION_STRING` - Your Azure AI Foundry project connection string (from the overview page)
   - `MODEL_DEPLOYMENT_NAME` - The name of your deployed model in Azure AI Foundry

## High-Level Flow
We'll do the following:
1. **Create** an AI Search index programmatically with sample fitness data.
2. **Upload** documents (fitness items) to the index.
3. **Create** an Agent that references our new index using `AzureAISearchTool`.
4. **Run queries** to see how it fetches from the index.
 
 <img src="./seq-diagrams/5-ai-search.png" width="30%"/>


## 1. Create & Populate Azure AI Search Index
We'll create a minimal index called `myfitnessindex`, containing a few example items.
Make sure to set your environment variables for `SEARCH_ENDPOINT` and `SEARCH_API_KEY`. We'll use the `azure.search.documents.indexes` classes to manage the index schema. We'll also upload some sample data.


In [ ]:
import os
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import SearchIndex, SimpleField, SearchFieldDataType
from azure.search.documents import SearchClient

search_endpoint = os.environ.get("SEARCH_ENDPOINT")
search_api_key = os.environ.get("SEARCH_API_KEY")

index_name = "myfitnessindex"

try:
    credential = AzureKeyCredential(search_api_key)
    index_client = SearchIndexClient(endpoint=search_endpoint, credential=credential)
    print("✅ SearchIndexClient created")
except Exception as e:
    print(f"❌ Error creating SearchIndexClient: {e}")

**Define the index** schema with a `FitnessItemID` key and a few fields to store product info.


In [ ]:
def create_fitness_index():
    fields = [
        SimpleField(name="FitnessItemID", type=SearchFieldDataType.String, key=True),
        SimpleField(name="Name", type=SearchFieldDataType.String, filterable=True, searchable=True),
        SimpleField(name="Category", type=SearchFieldDataType.String, filterable=True, facetable=True, searchable=True),
        SimpleField(name="Price", type=SearchFieldDataType.Double, filterable=True, sortable=True, facetable=True),
        SimpleField(name="Description", type=SearchFieldDataType.String, searchable=True)
    ]

    index = SearchIndex(name=index_name, fields=fields)

    # If index already exists, we can delete first to start fresh (optional)
    if index_name in [x.name for x in index_client.list_indexes()]:
        index_client.delete_index(index_name)
        print(f"🗑️ Deleted existing index: {index_name}")

    # Create brand-new index
    created = index_client.create_index(index)
    print(f"🎉 Created index: {created.name}")

# Create the index
create_fitness_index()

**Upload some sample documents** to `myfitnessindex`. We'll add a few items for demonstration.


In [ ]:
def upload_fitness_docs():
    # Construct a search client for data upload
    search_client = SearchClient(endpoint=search_endpoint,
                                 index_name=index_name,
                                 credential=AzureKeyCredential(search_api_key))

    sample_docs = [
        {
            "FitnessItemID": "1",
            "Name": "Adjustable Dumbbell",
            "Category": "Strength",
            "Price": 59.99,
            "Description": "A compact, adjustable weight for targeted muscle workouts."
        },
        {
            "FitnessItemID": "2",
            "Name": "Yoga Mat",
            "Category": "Flexibility",
            "Price": 25.0,
            "Description": "Non-slip mat designed for yoga, Pilates, and other exercises."
        },
        {
            "FitnessItemID": "3",
            "Name": "Treadmill",
            "Category": "Cardio",
            "Price": 499.0,
            "Description": "A sturdy treadmill with adjustable speed and incline settings."
        },
        {
            "FitnessItemID": "4",
            "Name": "Resistance Bands",
            "Category": "Strength",
            "Price": 15.0,
            "Description": "Set of colorful bands for light to moderate resistance workouts."
        }
    ]

    # Upload in one go
    result = search_client.upload_documents(documents=sample_docs)
    print(f"🚀 Upload result: {result}")

upload_fitness_docs()

### Verify the documents via a basic query
Let's do a quick search for **Strength** items.


In [ ]:
# Quick check
search_client = SearchClient(endpoint=search_endpoint,
                             index_name=index_name,
                             credential=AzureKeyCredential(search_api_key))

results = search_client.search(query="Strength", filter=None, top=10)
for doc in results:
    print(doc)


## 2. Create Agent With AI Search Tool
We'll create a new agent and attach an `AzureAISearchTool` referencing **myfitnessindex**.
In your environment, you need:
- `PROJECT_CONNECTION_STRING` - from your AI Foundry project overview
- `MODEL_DEPLOYMENT_NAME` - from the deployed model name

Let's initialize the `AIProjectClient` with `DefaultAzureCredential`.


In [ ]:
import os
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import AzureAISearchTool, ConnectionType

try:
    project_client = AIProjectClient.from_connection_string(
        credential=DefaultAzureCredential(),
        conn_str=os.environ["PROJECT_CONNECTION_STRING"],
    )
    print("✅ Successfully initialized AIProjectClient")
except Exception as e:
    print(f"❌ Error initializing project client: {e}")

### Find or create the Azure AI Search connection within your Foundry project
If you've already created a connection for your Azure AI Search resource in the Foundry project, you can list them and pick the correct one. If not, you can create a new connection in the [Portal or CLI usage for Foundry](https://learn.microsoft.com/azure/).


In [ ]:
conn_id = None
all_connections = project_client.connections.list()
for c in all_connections:
    if c.connection_type == ConnectionType.AZURE_AI_SEARCH:
        conn_id = c.id
        print(f"Found existing Azure AI Search connection: {conn_id}")
        break

if not conn_id:
    print("❌ No Azure AI Search connection found in your project.\n",
          "Please create one or ask your admin to do so.")

### Create the Agent with `AzureAISearchTool`
We'll attach the tool, specifying the index name we created.


In [ ]:
# Replace with your model deployment name from environment
model_name = os.environ.get("MODEL_DEPLOYMENT_NAME", "my-model-deployment")

agent = None
if conn_id:
    # Initialize the search tool with your index
    ai_search_tool = AzureAISearchTool(
        index_connection_id=conn_id,
        index_name=index_name  # our myfitnessindex
    )

    # Create the agent
    with project_client:
        agent = project_client.agents.create_agent(
            model=model_name,
            name="fitness-agent-search",
            instructions="""
            You are a Fitness Shopping Assistant. You help users find items, but always disclaim not to provide medical advice.
            """,
            tools=ai_search_tool.definitions,
            tool_resources=ai_search_tool.resources,
            headers={"x-ms-enable-preview": "true"},
        )
        print(f"🎉 Created agent, ID: {agent.id}")

## 3. Run a Conversation with the Agent
We'll open a new thread, post a question, and let the agent search the index for relevant items.


In [ ]:
def run_agent_query(question: str):
    # Create a new thread
    thread = project_client.agents.create_thread()
    print(f"📝 Created thread, ID: {thread.id}")

    # Create a user message
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content=question
    )
    print(f"💬 Created user message, ID: {message.id}")

    # Create and process agent run
    run = project_client.agents.create_and_process_run(
        thread_id=thread.id,
        assistant_id=agent.id
    )
    print(f"🤖 Agent run status: {run.status}")

    if run.last_error:
        print("⚠️ Run error:", run.last_error)

    # Retrieve all messages in the thread
    msg_list = project_client.agents.list_messages(thread_id=thread.id)
    # We'll print the assistant's last reply
    for m in reversed(msg_list.data):
        if m.role == "assistant" and m.content:
            print("\nAssistant says:")
            for c in m.content:
                if hasattr(c, "text"):
                    print(c.text.value)
            break

# Let's try some queries
if agent:
    run_agent_query("Which items are good for strength training?")
    run_agent_query("I need something for cardio under $300, any suggestions?")

## 4. Cleanup
We'll clean up the agent. (In production, you might want to keep it!)


In [ ]:
if agent:
    project_client.agents.delete_agent(agent.id)
    print("🗑️ Deleted agent")

index_client.delete_index(index_name)
print(f"🗑️ Deleted index {index_name}")

# 🎉 Congrats!
You've successfully:
1. **Created** an Azure AI Search index programmatically.
2. **Populated** it with sample fitness data.
3. **Created** an Agent that queries the index using `AzureAISearchTool`.
4. **Asked** the agent for item recommendations.

Continue exploring how to integrate **OpenTelemetry** or the `azure-ai-evaluation` library for advanced tracing and evaluation capabilities. Have fun, and stay fit! 🏆
